# 🔧 OpenCode Compatible LLM Server

**Uses Qwen 2.5 Coder - the ONLY Ollama model with reliable tool support.**

> ⚠️ DeepSeek, Mistral, CodeLlama do NOT properly support function calling through Ollama.

In [ ]:
#@title 📥 Install
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('✅ Ready')

In [ ]:
#@title 🤖 Start Model
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
# Qwen is the only model with reliable tool support
!ollama pull qwen2.5-coder:7b
print('\n✅ Model ready')

In [ ]:
#@title 🚀 API Server (With Forced Tool Output)
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading, re

app = Flask(__name__)
MODEL = "qwen2.5-coder:7b"

# Force tool usage for action requests
TOOL_PROMPT = '''You are an AI coding assistant that uses tools to complete tasks.

## CRITICAL: When to use tools
For ANY request to create, edit, write, or modify files - you MUST respond with a tool call.

## Tool Call Format
When using a tool, respond ONLY with this JSON (no other text):
```json
{"name": "tool_name", "arguments": {"param": "value"}}
```

## Available Tools
- write: Write content to a file. Args: path (string), content (string)
- edit: Edit existing file. Args: path (string), content (string)  
- read: Read a file. Args: path (string)
- bash: Run a command. Args: command (string)

## Examples
User: "Create hello.py with print hello world"
```json
{"name": "write", "arguments": {"path": "hello.py", "content": "print('hello world')"}}
```

User: "Run python hello.py"
```json
{"name": "bash", "arguments": {"command": "python hello.py"}}
```

## For Questions/Greetings
Respond naturally with text. Only use tools for file/command operations.'''

def needs_tools(msg):
    m = msg.lower() if msg else ''
    actions = ['create', 'write', 'make', 'generate', 'new', 'save', 'add',
               'edit', 'modify', 'update', 'change', 'fix', 'refactor',
               'delete', 'remove', 'run', 'execute', 'test', 'build',
               'file', 'script', 'code', 'function', 'class']
    return any(a in m for a in actions)

def get_valid_tools(tools):
    return {t['function']['name'] for t in tools or [] if t.get('type') == 'function'}

def extract_tool(text, valid):
    if not text:
        return None
    
    # Try code block
    m = re.search(r'```(?:json)?\s*([\s\S]*?)```', text)
    if m:
        try:
            d = json.loads(m.group(1).strip())
            name = d.get('name', '')
            if name in valid or not valid:  # Accept if valid list empty or matches
                args = d.get('arguments', {})
                return name, json.dumps(args) if isinstance(args, dict) else args
        except: pass
    
    # Try raw JSON
    m = re.search(r'\{\s*"name"\s*:\s*"([^"]+)"\s*,\s*"arguments"\s*:\s*(\{[^{}]*\})', text)
    if m:
        name = m.group(1)
        if name in valid or not valid:
            return name, m.group(2)
    
    return None

@app.route('/v1/models', methods=['GET'])
def list_models():
    return jsonify({"object": "list", "data": [{"id": MODEL, "object": "model"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    tools = data.get('tools', [])
    valid_tools = get_valid_tools(tools)
    
    # Get last user message
    user_msg = ''
    for m in reversed(messages):
        if m.get('role') == 'user' and m.get('content'):
            user_msg = str(m['content'])
            break
    
    use_tools = needs_tools(user_msg)
    print(f"[{time.strftime('%H:%M:%S')}] '{user_msg[:40]}' tools={use_tools}")
    
    # Build messages
    msgs = [{'role': 'system', 'content': TOOL_PROMPT}]
    for m in messages:
        if m.get('role') != 'system':
            msgs.append(m)
    
    # Call Ollama directly with tools
    try:
        payload = {
            'model': MODEL,
            'messages': msgs,
            'stream': False,
            'options': {'num_ctx': 8192}
        }
        if use_tools and tools:
            payload['tools'] = tools
        
        r = req.post('http://localhost:11434/api/chat', json=payload, timeout=120)
        result = r.json()
        content = result.get('message', {}).get('content', '')
        native_tools = result.get('message', {}).get('tool_calls', [])
    except Exception as e:
        print(f"Error: {e}")
        content = "I'm ready to help! What would you like me to do?"
        native_tools = []
    
    # If Ollama returned native tool calls
    if native_tools:
        formatted = []
        for tc in native_tools:
            formatted.append({
                "id": f"call_{uuid.uuid4().hex[:8]}",
                "type": "function",
                "function": {
                    "name": tc.get('function', {}).get('name', ''),
                    "arguments": json.dumps(tc.get('function', {}).get('arguments', {}))
                }
            })
        print(f"  → Native tool: {formatted[0]['function']['name']}")
        return jsonify({
            "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
            "object": "chat.completion",
            "created": int(time.time()),
            "model": MODEL,
            "choices": [{"index": 0, "message": {"role": "assistant", "content": None, "tool_calls": formatted}, "finish_reason": "tool_calls"}],
            "usage": {"prompt_tokens": 100, "completion_tokens": 50, "total_tokens": 150}
        })
    
    # Try to extract tool from text
    if use_tools and content:
        tool = extract_tool(content, valid_tools)
        if tool:
            name, args = tool
            print(f"  → Parsed tool: {name}")
            return jsonify({
                "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
                "object": "chat.completion",
                "created": int(time.time()),
                "model": MODEL,
                "choices": [{"index": 0, "message": {"role": "assistant", "content": None, "tool_calls": [{"id": f"call_{uuid.uuid4().hex[:8]}", "type": "function", "function": {"name": name, "arguments": args}}]}, "finish_reason": "tool_calls"}],
                "usage": {"prompt_tokens": 100, "completion_tokens": 50, "total_tokens": 150}
            })
    
    # Return text (clean up any stray JSON)
    if content:
        content = re.sub(r'```json[\s\S]*?```', '', content).strip()
        content = re.sub(r'\{\s*"name"[^}]+\}', '', content).strip()
    if not content:
        content = "Hello! I'm ready to help. What would you like me to do?"
    
    print(f"  → Text ({len(content)} chars)")
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": MODEL,
        "choices": [{"index": 0, "message": {"role": "assistant", "content": content}, "finish_reason": "stop"}],
        "usage": {"prompt_tokens": 100, "completion_tokens": len(content.split()), "total_tokens": 100 + len(content.split())}
    })

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print(f'\n✅ Server running!')

In [ ]:
#@title 🧪 Test
import requests

# Test tool call
print("Testing: Create hello.py...")
r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Create a file called hello.py that prints hello world'}],
    'tools': [{'type': 'function', 'function': {'name': 'write', 'description': 'Write file', 'parameters': {'type': 'object', 'properties': {'path': {'type': 'string'}, 'content': {'type': 'string'}}}}}]
}, timeout=120)

resp = r.json()['choices'][0]
if resp['message'].get('tool_calls'):
    print(f"✅ SUCCESS! Tool call: {resp['message']['tool_calls'][0]['function']}")
else:
    print(f"❌ Got text instead: {resp['message'].get('content', 'empty')[:100]}")

In [ ]:
#@title 🌐 Start Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''
            <div style="background:linear-gradient(135deg,#667eea,#764ba2);padding:30px;border-radius:20px">
                <h2 style="color:white;margin:0">🚀 OpenCode Ready!</h2>
                <p style="color:white;font-size:20px;font-family:monospace;margin:15px 0">{url}/v1</p>
                <p style="color:#ddd">Model: qwen2.5-coder:7b (with tool support)</p>
            </div>
            '''))
            break

for line in tunnel.stdout:
    print(line, end='')